In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pandas as pd
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
token_dict = pd.DataFrame(tk_dbrt.vocab.items(), columns=["token", "id"]).sort_values("id").reset_index(drop=True).set_index("id")

texto = "No Hola a todos"
tokens = tokenizer.tokenize(texto)
ids = tokenizer.convert_tokens_to_ids(tokens)

token_dict.loc[ids]

In [ ]:
from src.datasets.text_datasets.old.tripadvisor.W2Vdataset import W2Vdataset
from src.models.text_models.W2V import W2V

base_path = "/media/nas/datasets/tripadvisor/restaurants/"
cities = ["gijon", "barcelona", "madrid"]

remove_numbers = True
remove_accents = True
remove_plurals = False
lemmatization = True
stemming = False

w2v_dimen = 128
seed = 100
gpu = 1

w2v_dts = W2Vdataset({"cities": cities, "city": "multi", "seed": seed, "data_path": base_path, "save_path": "data/",  # base_path + "Datasets/",
                        "remove_plurals": remove_plurals, "stemming": stemming, "lemmatization": lemmatization,
                        "remove_accents": remove_accents, "remove_numbers": remove_numbers,
                        }, load=[])
w2v_dts.CONFIG["dataset"]="multi"
w2v_dts.CONFIG["subset"]="-".join(cities)

In [ ]:
w2v_dts.DATA

In [ ]:
w2v_mdl = W2V({"model": {"train_set": "ALL_TEXTS", "min_count": 100, "window": 5, "n_dimensions": w2v_dimen, "seed": seed},
                "session": {"gpu": gpu, "mixed_precision": True, "in_md5": False}}, w2v_dts)

w2v_mdl.train()

In [ ]:
# Imprimir los tokens
for token, idt in tk_dbrt.vocab.items():
    print(idt,token)

In [ ]:
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow as tf



In [ ]:
from transformers import AutoTokenizer, TFAutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")
model = TFAutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish")


In [ ]:
text = "Hola mundo"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
output[1]

In [ ]:
encoded_input

In [ ]:
preprocessor = hub.load( "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

# Step 1: tokenize batches of text inputs.
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string) # This SavedModel accepts up to 2 text inputs.
tokenize = hub.KerasLayer(preprocessor.tokenize)
tokenized_inputs = [tokenize(text_input)]

# Step 2: pack input sequences for the Transformer encoder.
seq_length = 200  # Your choice here.
bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs, arguments=dict(seq_length=seq_length))  # Optional argument.
encoder_inputs = bert_pack_inputs(tokenized_inputs)

# Model
encoder = hub.KerasLayer( "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 128].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].

In [ ]:
embedding_model = tf.keras.Model(text_input, sequence_output)
sentences = tf.constant(["Hello world"])

res = embedding_model(sentences)

## Spacy dependency parse

In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

In [21]:
# Frase de ejemplo
sentence = "Pizza with cheese but no tomato."

# Procesar la frase con el modelo
doc = nlp(sentence)

# Generar el gráfico de dependencia
displacy.render(doc, style='dep', jupyter=True, minify=True)


In [ ]:

for token in doc:
    if token.dep_ != 'ROOT':  # Ignorar la raíz del árbol de dependencias
        head = token.head.text
        relation = token.dep_
        child = token.text
        print(f'{head} --{relation}--> {child}')


In [20]:

# Construir el diccionario de relaciones
relations_dict = {}
for token in doc:
    if token.dep_ != 'ROOT':  # Ignorar la raíz del árbol de dependencias
        head = token.head.text
        relation = token.dep_
        child = token.text
        
        # Agregar la relación al diccionario
        if head in relations_dict:
            relations_dict[head].append((relation, child))
        else:
            relations_dict[head] = [(relation, child)]

# Imprimir el diccionario de relaciones
for word, relations in relations_dict.items():
    print(word)
    for relation in relations:
        rel, child = relation
        print(f'--{rel}--> {child}')
    print()


Pizza
--prep--> with
--cc--> but
--conj--> tomato
--punct--> .

with
--pobj--> cheese

tomato
--det--> no

